In [ ]:
#First install the necesary libraries
!pip install openai==1.1.1
!pip install panel

In [ ]:
import openai
import geopy
import panel as pn
#from mykeys import openai_api_key
openai.api_key="your-api-key"

In [ ]:
def continue_conversation(messages, temperature=0.7):
    response = openai.chat.completions.create(
        model="gpt-4-0613",
        messages=messages,
        temperature=temperature,
    )
    # print(response)
    #print(str(response.choices[0].message["content"]))
    answer = response.choices[0].message.content
    # print(answer)
    return answer

In [ ]:
pn.extension()

panels = []

client_prompt = pn.widgets.TextInput(value="Hi", placeholder='Enter text here…')
button_conversation = pn.widgets.Button(name="talk")

In [ ]:
# context = [ {'role':'system', 'content':"""
# You are a restaurant discovery chatbot. your job is to help people find restaurants and cafe.
# Here are the lists of tasks you have to perform.
# 1. Identify the intent of the customer. We only support Greet and Restaurant discovery. In case of any other respond with apology.
# 2. Once the customer starts with restaurant intent, identify the entities required for restaurant search.
# 3. You have to fill these slots for restaurant search -- [cuisine, budget, number of tables]. For cafe only ask for [timing, budget, number of people]. If the customer doesnt provide these info, ask them 1 by one.
# 4. Once you have all the 3 details, write a sql query assuming a simple table restaurant, sort them by rating.
# """} ]

In [ ]:
# def identify_user_intent(user_query):
#     prompt = f"Identify the intent of the user. Possible intents: [RestaurantDiscovery, HotelBooking, FlightBooking, Other]. User Query: {user_query}"

#     # Using GPT-3.5 to generate intent based on user query and prompt
#     response = openai.Completion.create(
#         engine="text-davinci-002",  # or specify the appropriate engine
#         prompt=prompt,
#         max_tokens=50,
#         temperature=0.5,
#         stop=None,
#     )

#     # Extracting the generated intent from the model's response
#     generated_intent = response.choices[0].text.strip()

#     # Mapping generated intent to predefined intents
#     predefined_intents = ["RestaurantDiscovery", "HotelBooking", "FlightBooking", "Other"]
#     matched_intent = next((intent for intent in predefined_intents if intent.lower() in generated_intent.lower()), "Unknown")

#     return matched_intent

In [ ]:
# user_intent = identify_user_intent(user_query)

In [ ]:
# system_prompt = f"""You are a general-purpose discovery chatbot. Your goal is to assist users with by finding various options as per need and assist in bookings.
# First you try to find intent of user. Once user intent is found, identify entities required for search and fill those slots.
# If user doesn't provide them, ask it to user one by one.
# Also we can not do hospital bookings.
# Intent:mRestrunt Booking; Entities: ['budget', 'no. of people', 'food type']

# Intent:Hospital Booking; Entities: ['budget', 'no. of people', 'food type']
# Once you have all the details, write a sql query assuming a simple booking, and sort them by rating.
# """
system_prompt = f"""
You are a discovery chatbot. Your goal is to assist users with by finding various options as per need and assist in bookings.
First you try to find intent of user. Once user intent is found, identify entities required for search and fill those slots.
If user doesn't provide them, ask it to user one by one.
We only help user for specific intents only.
Once you have all the details, write a sql query assuming a simple booking, and sort them by the factor as specified. Make sure that in sql query, you use same names as entities provided below for the intent.

Intent: Restraunt booking; Entities:[cuisine, budget, number of tables]; Sort by: rating
Intent: Flight ticket booking; Entities: [from, to, date]; Sort by: rating

"""
context = [{'role': 'system', 'content': system_prompt}]

In [ ]:
# #Creating the prompt
# #read and undertand it.
# def start_conversation(user_query):
#     prompt1 = "Check the intent of the user. Right now we support only these 2 intents [Greet, Restaurant Discovery, Others]. Response - Intent."
#     context = [ {'role':'system', 'content':prompt1+"----"+user_query} ]
#     response = continue_conversation(context)
#     return response

# user_query = "Hi there. I would like to search for some good restaurants around."
# response1 = start_conversation(user_query)
# response1

In [ ]:
# if response1 == "Greet":
#     prompt2 = "Ask customer for queries polietly."
# if response1 == "Others":
#     prompt2 = "Polietly say that we support only restaurant discovery."

# # if restaurant discovery
# slots = ["cuisine", "budget", "discount"]

# context = [ {'role':'system', 'content':prompt2} ]
# response2 = continue_conversation(context)
# response2

In [ ]:
def add_prompts_conversation(_):
    # Get the value introduced by the user
    prompt = client_prompt.value_input
    client_prompt.value = ''

    # Append to the context the User promnopt.
    context.append({'role':'user', 'content':f"{prompt}"})

    # Get the response.
    response = continue_conversation(context)

    # Add the response to the context.
    context.append({'role':'assistant', 'content':f"{response}"})

    # Undate the panels to shjow the conversation.
    panels.append(
        pn.Row('User:', pn.pane.Markdown(prompt, width=600)))
    panels.append(
        pn.Row('Assistant:', pn.pane.Markdown(response, width=600)))
    # print(context)
    return pn.Column(*panels)

In [ ]:
interactive_conversation = pn.bind(add_prompts_conversation, button_conversation)

dashboard = pn.Column(
    client_prompt,
    pn.Row(button_conversation),
    pn.panel(interactive_conversation, loading_indicator=True),
)

dashboard